Trabajo basado en el paper **Multi-hop assortativities for network classification** por <em>Leonardo Gutierrez-Gomez</em> y <em>Jean-Charles Delvenne</em>, los datos de prueba se encuentran en [Nino Shervashidze](https://members.cbio.mines-paristech.fr/~nshervashidze/code/)

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scipy.io
import scipy.sparse.linalg as sp_linalg
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.preprocessing import OneHotEncoder

![eq_3_2](images/eq_3_2.png)

In [2]:
#Ecuacion 3.2: calcula la autocovarianza de matrices para un camino aleatorio en un momento t
#t: momento a calcular
#P: matriz diagonal de distribución estacionaria
#p: vector de distribución estacionaria
#Me: matriz de markov
def ro(t, P, p, Me):
    return P * np.linalg.matrix_power(Me, t) - pi.transpose() * pi

![eq_3_3.png](images/eq_3_3.png)

In [3]:
#Ecuacion 3.3: covarianza del atributo v para los primeros t tiempos determinados por lag 
#v: atributos de cada nodo (arreglo de números)
#cova: arreglo de covarianzas para los primeros t tiempos
#lag: numero de tiempos a calcular
def attr_cov(v, cova, lag):
    cov = []
    for t in range(lag+1):
        cov.append((v.transpose() * cova[t] * v).tolist()[0][0])
    return cov

![eq_3_4.png](images/eq_3_4.png)

![eq_3_6.png](images/eq_3_6.png)

In [4]:
#Ecuacion 3.4: autocovarianza de variable categorica para los primeros t tiempos determinados por lag
#Ecuacion 3.6: covarianza de variable categorica para los primeros t tiempos determinados por lag
#Me: matriz de markov
#p: vector de distribución estacionaria
#P: matriz diagonal de distribución estacionaria
#Ha: matriz one hot de la variable categorica
#lag: numero de tiempos a calcular
def covarianza(Me, p, P, Ha, lag): 
    # cov son las matrices de covarianza
    # aut son las matrices de autocovarianza
    # tr son las trazas de aut
    cov = []
    aut = []
    tr = []
    for t in range(lag +1):
        cov.append(ro(t, P, p, Me))
        aut.append(Ha.transpose() * cov[-1] * Ha)
        tr.append(np.trace(aut[-1]))
    return cov, aut, tr

In [5]:
#genera los atributos de distribucion estacionaria del grafo (vector y matriz diagonal)
#graph: grafo de networkx
#degrees: arreglo de grados
def distribucion_estacionaria(graph, degrees):
    m = len(graph.edges)
    #pi es el vector de distribución estacionaria
    p = degrees.transpose() / (2*m)
    #PI es la matriz diagonal de distribución estacionaria
    P = np.diag(p.tolist()[0])
    return p, P

In [6]:
#Calcula el promedio de los labels de los nodos
#Ha: matriz one hot de la variable categorica
#p: vector de distribución estacionaria
def avg_feats(Ha, p):
    avg = []
    for i in range(len(Ha[0])):
        avg.append(p * np.asmatrix(Ha[:,i]).transpose())
    return avg

In [7]:
#procesar los labels correspondientes al grafo, extrae los valores únicos y la codificación one hot
def process_labels(lab):
    unique = np.unique(lab.flatten())
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(unique.reshape(-1, 1))
    oneHot = enc.transform(lab.reshape(-1, 1)).toarray()
    return unique, oneHot

In [8]:
#Genera el grafo junto con sus principales características 
#(matriz de adjacencia, arreglo de grados, matriz de grados, matriz de markov)
#graph: representacion del grafo como matriz de adjacencia
def generar_grafo(graph):
    #Construccion del grafo
    G = nx.from_numpy_matrix(graph)
    #A es la matriz de adjacencia del grafo
    A = nx.adjacency_matrix(G).todense()
    #d es el arreglo de grados del grafo
    d = A * np.ones((G.number_of_nodes(), 1))
    #D es la matriz de grados diagonales
    D = np.diag(d.transpose().tolist()[0])
    #M es la matriz de markov del grafo
    M = np.linalg.inv(D) * A
    return G, A, d, M

![features](images/features.png)

In [9]:
#extrae los features del grafo
#graph: grafo de networkx
#degrees: arreglo de grados
#p: vector de distribución estacionaria
#P: matriz diagonal de distribución estacionaria
#lag: numero de tiempos a calcular
#evec: eigenvectors principales
#Me: matriz de markov
#lab: labels de los nodos
def extraccion_features(graph, degrees, p, P, lag, evec, Me, lab):
    #feature1: multi-hop assortativities of node IDs, setting H = I in Eq. (3.6) for 0, 1, ... , t hops
    #H es la matriz identidad del tamaño del grafo
    H = np.identity(len(degrees))
    cov, aut, res1 = covarianza(M, p, P, H, lag)
    #feature2: average of first p dominant left eigenvector of M, that is {πv : v is a dominant eigenvector of M}
    res2 = attr_cov(p.transpose(), cov, lag)
    #feature3: multi-hop assortativities of the first p dominant left eigenvectors of M (Eq. 3.3) for 0, 1, ... , t hops
    #segundo eigenvector
    pi2 = np.asmatrix(evec[:,1]).transpose()
    #normalizacion del segundo eigenvector
    h2 = pi2/np.linalg.norm(pi2, 1)
    res3 = attr_cov(h2, cov, lag)
    #tercer eigenvector
    pi3 = np.asmatrix(evec[:,2]).transpose()
    #normalizacion del tercer eigenvector
    h3 = pi3/np.linalg.norm(pi3, 1)
    res4 = attr_cov(h3, cov, lag)
    if node_labels:
        #extraccion de labels del grafo
        unique, categorias = process_labels(lab)
        cov, aut, tr = covarianza(Me, p, P, categorias, lag)
        #feature4: (If available) average of categorical metadata node attributes: {πhi : H = [h1, h2, ..hk ], 1 ≤ i ≤ k}
        res5 = np.array([np.diag(i) for i in aut]).flatten()
        #feature5: (If available) multi-hop assortativities of categorical metadata node attributes (Eq. 3.4) for 0, 1, ... , t hops
        res6 = avg_feats(categorias, p)
    #feature6: number of nodes
    res9 = graph.number_of_nodes()
    #feature7: number of edges
    res7 = graph.number_of_edges()
    r = {
        #f1: asortatividad de multiples saltos de los nodos
        0: res1,
        #f2: asortatividad de multiples saltos del primer vector propio dominante
        1: [i.real for i in res2],
        #f3: asortatividad de multiples saltos del segundo vector propio dominante
        2: [i.real for i in res3],
        #f4: asortatividad de multiples saltos del tercer vector propio dominante
        3: [i.real for i in res4]
    }
    if node_labels:
        #f5: promedio de los atributos de los nodos
        r[4] = res5
        #f6: asortatividad de multiples saltos para los atributos de los nodos
        r[5] = [el.tolist()[0][0] for el in res6]
    #f7: numero de arcos
    r[6] = res7
    #f8: promedio del primer vector propio dominante
    r[7] = (pi * pi.transpose()).tolist()[0][0]
    #f9: numero de nodos
    r[8] = res9
    #f10: promedio del segundo vector propio
    r[9] = (pi * pi2).tolist()[0][0].real
    #f11: promedio del tercer vector propio
    r[10] = (pi * pi3).tolist()[0][0].real
    
    return r

In [10]:
#guarda la informacion los features como una fila mas
#t: arreglo de titulos base
#features: features a agregar como nueva fila
#idx: indice de la fila actual
#d: mapa donde se almacenan los datos
def guardar_fila(t, features, idx, d):
    for title in range(len(t)):
        #confirmar si el feature se calculo
        if title in features:
            #confirmar que si el feature debe ser iterado para guardar sus elementos
            if type(features[title]) != float and type(features[title]) != int :
                for j in range(len(features[title])):
                    #generar titulo de la columna con el indice correspondiente
                    key = t[title] + '_' + str(j)
                    #si la columna no existe, se genera
                    if not key in d:
                        d[key] = []
                    curr_size = len(d[key])
                    #si la columna existe, pero no tiene los todas las filas, se llena de 0 los espacios entre la ultima fila y la actual
                    if  curr_size < i - 1:
                        d[key] = d[key] + [None for z in range(idx- curr_size)]
                    d[key].append(features[title][j])
            #si es un tipo primitivo, se guarda como es
            else:
                key = t[title]
                #si la columna no existe, se genera
                if not key in d:
                    d[key] = []
                d[key].append(features[title])

In [11]:
#ajustar los datos por si hay filas faltantes
#d: mapa que contiene los datos
#filas: numero esperado de filas
def ajustar_datos(d, filas):
    for key in d.keys():
        curr_size = len(d[key])
        #si hacen falta filas, se rellenan con 0
        if  curr_size < filas:
            d[key] = d[key] + [None for z in range(filas - curr_size)]
        #print(curr_size, len(d[key]))

In [12]:
titles = ['ID', 'pagerank', 'eigenvec_2nd_M', 'eigenvec_3rd_M', 'node_attribs', 'avg_node_labels', 'n_edges', 'avg_pi', 'avg_inv_pi', 'avg_eigenvec_2nd', 'avg_eigenvec_3rd']
#nombre del dataset
name = 'mutag'
#si utilizar los labels de los nodos
node_labels = True
#numero de tiempos a calcular
l = 3
#numero de vectors propios a utilizar
num_eig = 3

In [13]:
#carga del dataset
dataset = scipy.io.loadmat('datasets/'+name+'.mat')
names = list(dataset.keys())[3:]

In [14]:
data = {}
df = None
Graphs = dataset[names[1]][0]
num_graphs = len(Graphs)
for i in range(num_graphs):
    #print('procesando grafo en posicion', i)
    #labels de los nodos
    labels = Graphs[i][1][0][0][0].flatten()
    #total de nodos
    total = len(Graphs[i])
    #Construccion del grafo
    G, A, d, M = generar_grafo(Graphs[i][0])
    #calculo de distribucion estacionaria
    pi, PI = distribucion_estacionaria(G, d)
    #calculo de los primeros eigenvalues y egenvectors de mayor a menor magnitud
    evalues, evectors = sp_linalg.eigs(M.transpose(), k = num_eig, which = 'LM')
    r = extraccion_features(G, d, pi, PI, l, evectors, M, labels)
    guardar_fila(titles, r, i, data)
ajustar_datos(data, num_graphs)
df = pd.DataFrame(data)
df

,ID_0,ID_1,ID_2,ID_3,pagerank_0,pagerank_1,pagerank_2,pagerank_3,eigenvec_2nd_M_0,eigenvec_2nd_M_1,...,node_attribs_12,node_attribs_13,node_attribs_14,node_attribs_15,avg_node_labels_3,node_attribs_16,node_attribs_17,node_attribs_18,node_attribs_19,avg_node_labels_4
0,0.953361,-0.046639,0.379287,-0.046639,0.000111,0.000009,0.000039,-0.000003,0.002286,-0.002185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.954719,-0.045281,0.419005,-0.045281,0.000182,-0.000069,0.000083,-0.000033,0.002222,-0.001841,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.943182,-0.056818,0.375000,-0.056818,0.000176,-0.000035,0.000070,-0.000026,0.000176,-0.000035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.953361,-0.046639,0.379287,-0.046639,0.000111,0.000009,0.000043,-0.000001,0.002286,-0.002185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.936288,-0.063712,0.383657,-0.063712,0.000242,-0.000013,0.000096,-0.000020,0.004299,-0.004048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.896694,-0.103306,0.396694,-0.103306,0.000973,-0.000342,0.000285,-0.000143,0.015064,0.010308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,0.902778,-0.097222,0.402778,-0.097222,0.000965,-0.000482,0.000338,-0.000249,0.010417,-0.008970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,0.913265,-0.086735,0.377551,-0.086735,0.000586,-0.000143,0.000221,-0.000141,0.000586,-0.000143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186,0.915816,-0.084184,0.380102,-0.084184,0.000475,-0.000072,0.000171,-0.000092,0.007555,-0.007022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#generar csv
df.to_csv('features/' + name + '_features_py.csv', sep=',', header=True)